# ミニマム

https://docs.smith.langchain.com/

In [2]:
import openai
from langsmith import wrappers, traceable

# Auto-trace LLM calls in-context
client = wrappers.wrap_openai(openai.Client())

@traceable # Auto-trace this function
def pipeline(user_input: str):
    result = client.chat.completions.create(
        messages=[{"role": "user", "content": user_input}],
        model="gpt-4o-mini"
    )
    return result.choices[0].message.content

pipeline("lang smith test")
# Out:  Hello there! How can I assist you today?

'The "Lang Smith Test" does not refer to any widely recognized concept or test in psychology, science, or medicine that I am aware of as of my last knowledge update in October 2023. It\'s possible that you might be referring to a specific assessment tool or evaluation that is not well-known or is used in a specialized field.\n\nIf you can provide more context or clarify what you mean by "Lang Smith Test," I would be happy to help with information or related topics!'

# function call

In [8]:
assistant = client.beta.assistants.create(
    name="Math Tutor",
    instructions="You are a personal math tutor. Write and run code to answer math questions.",
    tools=[{"type": "code_interpreter"}],
    model="gpt-4o-mini",
)
thread = client.beta.threads.create()
user_input = "I need to solve the equation `3x + 11 = 14`. Can you help me?"

@traceable
def function_call_wrap(assistant, thread, user_input: str):
    message = client.beta.threads.messages.create(
        thread_id=thread.id,
        role="user",
        content=user_input
    )
    run = client.beta.threads.runs.create_and_poll(
        thread_id=thread.id,
        assistant_id=assistant.id,
    )
    return run

run = function_call_wrap(assistant, thread, user_input)

if run.status == "completed":
    messages = client.beta.threads.messages.list(thread_id=thread.id)
    print(messages.data[0].content[0].text.value)
else:
    print(run.status)

The solution to the equation \(3x + 11 = 14\) is \(x = 1\).


# Assistant function call

In [11]:
from typing_extensions import override
from openai import AssistantEventHandler
import json

assistant = client.beta.assistants.create(
    instructions="あなたは店舗運営とデータ分析のスペシャリストです。店舗管理者の相談に答えてください。",
    model="gpt-4o-mini",
    tools=[
        {
            "type": "function",
            "function": {
                "name": "fetch_store_ratings",
                "description": "Fetch store ratings for all stores. Returns: dict: A dictionary where keys are store IDs and values are ratings (1.0 to 5.0).",
                "parameters": {
                    "type": "object",
                    "properties": {},
                },
            },
        },
        {
            "type": "function",
            "function": {
                "name": "fetch_store_data",
                "description": "Fetch store information for the specified store ID.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "store_id": {
                            "type": "integer",
                            "description": "店舗ID",
                        }
                    },
                    "required": ["store_id"],
                },
            },
        },
    ],
)
thread = client.beta.threads.create()

message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="評価が最も高い店舗を調査し、その店舗名を教えてください。",
)

class EventHandler(AssistantEventHandler):
    @override
    def on_event(self, event):
        # Retrieve events that are denoted with 'requires_action'
        # since these will have our tool_calls
        if event.event == "thread.run.requires_action":
            run_id = event.data.id  # Retrieve the run ID from the event data
            self.handle_requires_action(event.data, run_id)

    @traceable
    def handle_requires_action(self, data, run_id):
        tool_outputs = []

        for tool in data.required_action.submit_tool_outputs.tool_calls:
            args = json.loads(tool.function.arguments)
            if tool.function.name == "fetch_store_ratings":
                output = {
                    "1": 1.3,
                    "2": 4.0,
                    "3": 1.0,
                    "4": 3.5,
                    "5": 3.6,
                    "6": 3.9,
                    "7": 3.5,
                    "8": 4.9,
                    "9": 3.4,
                    "10": 2.0,
                }
                output = json.dumps(output)
                tool_outputs.append({"tool_call_id": tool.id, "output": output})
            elif tool.function.name == "fetch_store_data":
                output = {"id": 8, "name": "モブレストラン", "locale": "東京"}
                output = json.dumps(output)
                tool_outputs.append({"tool_call_id": tool.id, "output": output})

        # Submit all tool_outputs at the same time
        self.submit_tool_outputs(tool_outputs, run_id)

    @traceable
    def submit_tool_outputs(self, tool_outputs, run_id):
        # Use the submit_tool_outputs_stream helper
        with client.beta.threads.runs.submit_tool_outputs_stream(
            thread_id=self.current_run.thread_id,
            run_id=self.current_run.id,
            tool_outputs=tool_outputs,
            event_handler=EventHandler(),
        ) as stream:
            for text in stream.text_deltas:
                print(text, end="", flush=True)
            print()


with client.beta.threads.runs.stream(
    thread_id=thread.id, assistant_id=assistant.id, event_handler=EventHandler()
) as stream:
    stream.until_done()

評価が最も高い店舗は「ムブレストラン」で、評価は4.9です。



In [15]:
from typing_extensions import override
from openai import AssistantEventHandler
from langchain.callbacks import get_openai_callback
import json

assistant = client.beta.assistants.create(
    instructions="あなたは店舗運営とデータ分析のスペシャリストです。店舗管理者の相談に答えてください。",
    model="gpt-4o-mini",
    tools=[
        {
            "type": "function",
            "function": {
                "name": "fetch_store_ratings",
                "description": "Fetch store ratings for all stores. Returns: dict: A dictionary where keys are store IDs and values are ratings (1.0 to 5.0).",
                "parameters": {
                    "type": "object",
                    "properties": {},
                },
            },
        },
        {
            "type": "function",
            "function": {
                "name": "fetch_store_data",
                "description": "Fetch store information for the specified store ID.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "store_id": {
                            "type": "integer",
                            "description": "店舗ID",
                        }
                    },
                    "required": ["store_id"],
                },
            },
        },
    ],
)
thread = client.beta.threads.create()

message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="評価が最も高い店舗を調査し、その店舗名を教えてください。",
)

class EventHandler(AssistantEventHandler):
    @override
    def on_event(self, event):
        if event.event == "thread.run.requires_action":
            run_id = event.data.id
            self.handle_requires_action(event.data, run_id)

    @traceable
    def handle_requires_action(self, data, run_id):
        tool_outputs = []

        for tool in data.required_action.submit_tool_outputs.tool_calls:
            args = json.loads(tool.function.arguments)  # ツールの引数をJSONとしてパース
            print(f"APIリクエスト: {tool.function.name}, 引数: {args}")  # リクエストを出力

            # ダミーデータを生成
            if tool.function.name == "fetch_store_ratings":
                output = {
                    "1": 1.3,
                    "2": 4.0,
                    "3": 1.0,
                    "4": 3.5,
                    "5": 3.6,
                    "6": 3.9,
                    "7": 3.5,
                    "8": 4.9,
                    "9": 3.4,
                    "10": 2.0,
                }
                output_json = json.dumps(output)
                print(f"APIレスポンス: {tool.function.name}, 出力: {output}")  # レスポンスを出力
                tool_outputs.append({"tool_call_id": tool.id, "output": output_json})
            elif tool.function.name == "fetch_store_data":
                output = {"id": 8, "name": "モブレストラン", "locale": "東京"}
                output_json = json.dumps(output)
                print(f"APIレスポンス: {tool.function.name}, 出力: {output}")  # レスポンスを出力
                tool_outputs.append({"tool_call_id": tool.id, "output": output_json})

        # ツール出力を送信
        self.submit_tool_outputs(tool_outputs, run_id)

    @traceable
    def submit_tool_outputs(self, tool_outputs, run_id):
        with client.beta.threads.runs.submit_tool_outputs_stream(
            thread_id=self.current_run.thread_id,
            run_id=self.current_run.id,
            tool_outputs=tool_outputs,
            event_handler=EventHandler(),
        ) as stream:
            for text in stream.text_deltas:
                print(text, end="", flush=True)
            print()


with client.beta.threads.runs.stream(
    thread_id=thread.id, assistant_id=assistant.id, event_handler=EventHandler()
) as stream:
    stream.until_done()

APIリクエスト: fetch_store_ratings, 引数: {}
APIレスポンス: fetch_store_ratings, 出力: {'1': 1.3, '2': 4.0, '3': 1.0, '4': 3.5, '5': 3.6, '6': 3.9, '7': 3.5, '8': 4.9, '9': 3.4, '10': 2.0}
APIリクエスト: fetch_store_data, 引数: {'store_id': 8}
APIレスポンス: fetch_store_data, 出力: {'id': 8, 'name': 'モブレストラン', 'locale': '東京'}
評価が最も高い店舗は「ムブレストラン」で、評価は4.9です。所在地は東京です。



In [16]:
type(assistant)
.venv/lib/python3.11/site-packages/openai/resources/beta/assistants.py

.venv/lib/python3.11/site-packages/openai/_resource.py

openai.types.beta.assistant.Assistant